# GEMINI Model

In [15]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [16]:
GOOGLE_API_KEY = os.environ.get('GOOGLE_GEMINI_API_KEY')


### Basic Call Model

In [17]:
# Import the package
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(api_key=GOOGLE_API_KEY, model="gemini-1.5-pro")

In [18]:
# Define the question to ask the language model
question = "What is the capital of USA?"

# Invoke the language model with the question
response = llm.invoke(question)

# Extract key components from the response
content = response.content  # AI's response text
total_tokens = response.usage_metadata["total_tokens"]  # Total tokens used in the response

# Print the results
print(f"Response: {content}")
print(f"Total Tokens Used: {total_tokens}")

Response: Washington, D.C.
Total Tokens Used: 15


### Basic Example: Prompt Template + LLm
The most fundamental and commonly used case involves linking a prompt template with a model.

In [24]:
from langchain_core.prompts import PromptTemplate
# Define the prompt template
template = 'What is the capital of {country}?'
prompt_template = PromptTemplate.from_template(template)

In [26]:
# Generate the prompt.
prompt = prompt_template.format(country="india")
prompt

'What is the capital of india?'

### Chain Creation
LCEL (LangChain Expression Language)
Here, we use LCEL to combine various components into a single chain.


In [29]:
prompt = PromptTemplate.from_template("Please explain {topic} in simple terms.")
# Combine the prompt and model into a chain
model = ChatGoogleGenerativeAI(api_key=GOOGLE_API_KEY, model="gemini-1.5-pro")

chain = prompt | model

In [32]:
input = {"topic": "The Principles of Learning in Artificial Intelligence Models"}

In [35]:
response = chain.invoke(input)

In [37]:
response

AIMessage(content='Imagine teaching a dog a trick. You use treats (rewards) and corrections to shape their behavior. AI learning is similar, but instead of a dog, it\'s a computer program, and instead of treats, it\'s data and algorithms. Here are the basic principles:\n\n1. **Supervised Learning (Learning with a Teacher):**  You show the AI lots of labeled examples, like pictures of cats labeled "cat" and dogs labeled "dog." The AI learns the patterns to identify new pictures as either "cat" or "dog."  Think of it like flashcards.\n\n2. **Unsupervised Learning (Learning without a Teacher):**  You give the AI a bunch of data *without* labels. It has to figure out the patterns and relationships on its own, like grouping similar customers together based on their purchase history.  Think of it like sorting a pile of LEGOs by color and shape without instructions.\n\n3. **Reinforcement Learning (Learning through Trial and Error):**  You let the AI try different actions in an environment and

In [38]:
# Request for Streaming Output
answer = chain.stream(input)

# Streaming Output
for token in answer:
    print(token.content, end="", flush=True)

Imagine training a dog. You wouldn't expect it to understand complex commands right away.  AI models learn similarly, building up knowledge gradually through different "training" methods. Here are some key principles:

* **Supervised Learning:**  Like showing your dog a treat and saying "sit," you give the AI labeled examples (input data with the correct answer).  It learns to connect the input with the desired output. Example: Showing an AI pictures of cats and dogs, labeled as such, so it can learn to identify them.

* **Unsupervised Learning:**  This is like letting your dog explore the park and figure things out on its own.  You give the AI unlabeled data and it finds patterns, relationships, and structures within the data itself.  Example: Grouping customers based on their purchasing habits without knowing their demographics beforehand.

* **Reinforcement Learning:** This is like rewarding your dog for good behavior. The AI learns through trial and error by interacting with an env

### Output Parser
An Output Parser is a tool designed to transform or process the responses from an AI model into a specific format. Since the model's output is typically provided as free-form text, an Output Parser is essential to convert it into a structured format or extract the required data.


In [58]:
from langchain_core.output_parsers import StrOutputParser


In [59]:
template = """
You are a seasoned English teacher with 10 years of experience. Please write an English conversation suitable for the given situation.  
Refer to the [FORMAT] for the structure.

#SITUATION:
{question}

#FORMAT:
- Dialogue in English:
- Explanation of the Dialogue: 
"""

# Generate the prompt using the PromptTemplate
prompt = PromptTemplate.from_template(template)


llm = ChatGoogleGenerativeAI(api_key=GOOGLE_API_KEY, model="gemini-1.5-pro")

output_parser = StrOutputParser()

In [49]:
chain = prompt | llm | output_parser

In [50]:
print(chain.invoke({"question": "I want to go to a restaurant and order food."}))


- **Dialogue in English:**

**Waiter:** Good evening, welcome to The Golden Spoon. Do you have a reservation?

**Customer:** Good evening. No, we don't.  A table for two, please.

**Waiter:**  Right this way.  (Leads them to a table) Here you are. Your menus. I'll be back in a moment to take your order.

**(A few minutes later)**

**Waiter:** Are you ready to order?

**Customer:** Yes, I think so.  I'll have the grilled salmon with the roasted vegetables, please.

**Waiter:** And for you, sir?

**Customer 2:** I'm having trouble deciding between the steak and the pasta primavera.  What would you recommend?

**Waiter:** The steak is very popular. It's a ribeye, cooked to your liking.

**Customer 2:** Okay, I'll go with the ribeye, medium-rare, please.

**Waiter:** Excellent choices.  And for sides?

**Customer:** Could I have a side salad with the vinaigrette dressing?

**Customer 2:** I'll have the mashed potatoes.

**Waiter:** Certainly.  Anything to drink?

**Customer:** Just water f

In [51]:
# Execute the completed Chain to obtain a response
# Request for Streaming Output
answer = chain.stream({"question": "I want to go to a restaurant and order food."})

# Streaming Output
for token in answer:
    print(token, end="", flush=True)

- **Dialogue in English:**

**Waiter:** Good evening, welcome to The Golden Spoon. Do you have a reservation?

**Customer:** Good evening. No, we don't.  A table for two, please.

**Waiter:** Right this way.  (Leads them to a table)  Here are your menus.  Can I get you anything to drink while you look?

**Customer:**  I'd like a glass of iced tea, please.

**Customer 2:** I'll have a sparkling water, no lemon.

**Waiter:** Certainly.  I'll be back with your drinks in a moment. (Leaves)

**(A few minutes later, the waiter returns with the drinks.)**

**Waiter:** Here you are. Are you ready to order?

**Customer:** Yes, I think so. I'll have the grilled salmon with the roasted vegetables.

**Customer 2:**  I'm having trouble deciding... Could you tell me what the soup of the day is?

**Waiter:**  The soup of the day is creamy tomato basil.

**Customer 2:**  Hmm, that sounds lovely.  I'll have that to start, and then the chicken Caesar salad, please.

**Waiter:**  Excellent choices.  The 